In [1]:
from utils import *
from sklearn.pipeline import Pipeline
# from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import pickle

# Load Data

In [2]:
params = {
        'na_data': 'fill',
        'drop_outlier_days': 300,
        'embed':True,
        'buckets':[-1,3,14,30,100,99999999],
        'sample_dict':
            {
            'stratify_col':'Type',
            'train_size':0.6, 'validate_size':0.2, 'test_size':0.2
            }
        }
train_df, validate_df, test_df = load_df(params)
train_df = train_df[train_df.dataset=='Sonoma']
validate_df = validate_df[validate_df.dataset=='Sonoma']
test_df = test_df[test_df.dataset=='Sonoma']

Intake Date is NOT A STRING
Outcome Date is NOT A STRING
replace null values in Name with 'Unknown'
replace null values in Date_Of_Birth with 'Unknown'
Date Of Birth is NOT A STRING
replace null values in Outcome_Type with 'Unknown'
replace null values in Intake_Subtype with 'Unknown'
replace null values in Outcome_Subtype with 'Unknown'
replace null values in Outcome_Condition with 'Unknown'
Intake Date is NOT A STRING
Outcome Date is NOT A STRING
Days in Shelter is NOT A STRING
Count is NOT A STRING
replace null values in Name with 'Unknown'
replace null values in Size with 'Unknown'
replace null values in Outcome_Type with 'Unknown'
replace null values in Outcome_Subtype with 'Unknown'
replace null values in Outcome_Condition with 'Unknown'
replace null values in Outcome_Jurisdiction with 'Unknown'
replace null values in Outcome_Zip_Code with 'Unknown'
replace null values in Location with 'Unknown'


/Users/jessicahuber/source/capstone2/load_Austin.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date Of Birth'].fillna('01/01/1900', inplace=True)


Date Of Birth is NOT A STRING
Intake Date is NOT A STRING
Outcome Date is NOT A STRING
Days in Shelter is NOT A STRING
replace null values in Name with 'Unknown'
replace null values in Outcome_Subtype with 'Unknown'
replace null values in Size with 'Unknown'
replace null values in Date_Of_Birth with 'Unknown'
replace null values in Impound_Number with 'Unknown'
replace null values in Kennel_Number with 'Unknown'
replace null values in Animal_ID with 'Unknown'
replace null values in Intake_Type with 'Unknown'
replace null values in Intake_Subtype with 'Unknown'
replace null values in Outcome_Condition with 'Unknown'
replace null values in Intake_Jurisdiction with 'Unknown'
replace null values in Outcome_Jurisdiction with 'Unknown'
replace null values in Outcome_Zip_Code with 'Unknown'
replace null values in Location with 'Unknown'
replace null values in Multiple_Visit_Count with 'np.nan'
replace null values in Age_inDays_at_Income with 'np.nan'
replace null values in Age_Group with 'Unk

/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# Create Preprocessor

In [3]:
preprocessor, X_train, y_train, X_test, y_test, label_encoder_transformer = sklearn_pipeline(train_df, validate_df)

# Create Pipeline

In [12]:
# Create the pipeline with the preprocessor and the RandomForestClassifier
best_params = {'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'subsample': 0.8}
XGBpipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(random_state=42, **best_params))
])

# Train Model

In [13]:
# Train the model
XGBpipeline.fit(X_train, y_train)
# Predict on the test data
y_pred = XGBpipeline.predict(X_test)
# Evaluate the model
print(classification_report(y_test, y_pred))

fit_transform
fit
transform
transform
              precision    recall  f1-score   support

           0       0.78      0.88      0.82      1894
           1       0.58      0.56      0.57      1301
           2       0.50      0.44      0.47       944
           3       0.62      0.60      0.61       884
           4       0.58      0.14      0.23        97

    accuracy                           0.66      5120
   macro avg       0.61      0.53      0.54      5120
weighted avg       0.64      0.66      0.65      5120



### Save Model Pipeline

In [14]:
with open('XGBpipeline.pkl', 'wb') as file:
    pickle.dump(XGBpipeline, file)

### Load Model Pipeline

In [15]:
with open('XGBpipeline.pkl', 'rb') as file:
    test_pipeline = pickle.load(file)

In [16]:
# Predict on the test data
y_pred = test_pipeline.predict(X_test)
# Evaluate the model
print(classification_report(y_test, y_pred))

transform
              precision    recall  f1-score   support

           0       0.78      0.88      0.82      1894
           1       0.58      0.56      0.57      1301
           2       0.50      0.44      0.47       944
           3       0.62      0.60      0.61       884
           4       0.58      0.14      0.23        97

    accuracy                           0.66      5120
   macro avg       0.61      0.53      0.54      5120
weighted avg       0.64      0.66      0.65      5120



# TEST Model with API

In [17]:
from datetime import datetime, timedelta
from sodapy import Socrata
client = Socrata("data.sonomacounty.ca.gov", None)

end_date_str = datetime.now().strftime("%Y-%m-%d")
start_date = datetime.now() - timedelta(days = 50)
start_date_str = start_date.strftime("%Y-%m-%d")
results = client.get(
    "924a-vesw",
    where=f"intake_date between '{start_date_str}' and '{end_date_str}'",
)
results_df = pd.DataFrame.from_records(results)
params = {
    'na_data': 'fill',
    'drop_outlier_days': False,
    'embed':True,
    'buckets':[-1,3,14,30,100,99999999],
    'sample_dict':
        {
        'stratify_col':'Type',
        'train_size':0.6, 'validate_size':0.2, 'test_size':0.2
        }
    }
results_df = load_df(params, data=results_df, split_data=False)

Date Of Birth is NOT A STRING
Intake Date is NOT A STRING
Outcome Date is NOT A STRING
replace null values in Name with 'Unknown'
replace null values in Outcome_Date with 'Unknown'
replace null values in Outcome_Type with 'Unknown'
replace null values in Outcome_Subtype with 'Unknown'
replace null values in Outcome_Condition with 'Unknown'
replace null values in Outcome_Jurisdiction with 'Unknown'
replace null values in Outcome_Zip_Code with 'Unknown'
replace null values in Location with 'Unknown'
replace null values in Age_Group with 'Unknown'


/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/Users/jessicahuber/anaconda3/envs/datasci-207/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [18]:
# Predict on the test data
y_pred = test_pipeline.predict(results_df)
y_pred

transform


array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 3, 0, 3, 0, 0, 1, 0, 0, 0,
       1, 1, 3, 3, 1, 2, 2, 2, 3, 1, 3, 3, 0, 0, 0, 0, 1, 0, 0, 3, 3, 0,
       0, 1, 0, 3, 3, 3, 1, 3, 0, 0, 3, 3, 3, 0, 0, 0, 0, 0, 0, 3, 3, 0,
       0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 3, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 3, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
       0, 2, 0, 0, 0, 0, 1, 2, 1, 3, 3, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 1, 0, 0,
       2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 3, 1, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       2, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 1, 0, 2, 2, 2, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 2, 0, 0, 0, 0, 1, 0, 1, 1, 1,